In [ ]:
import tensorflow
import mediapipe as mp
import cv2
import numpy as np
import sklearn
import matplotlib
import os

In [ ]:
# Initialize mediapipe elements: Drawing utils & Hand model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

## Landmarks Detection Function

In [ ]:
# Mediapipe Hands Detection
def hand_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    # By default opencv uses BGR
    image.flags.writeable = False

    results = model.process(image)    # Extract the landmarks using mp model
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)    # Convert to opencv compatible color space
    return image, results

In [ ]:
def draw_landmarks(image, result):
    if results.multi_hand_landmarks:    # If landmarks are recognized
        for num, hand in enumerate(results.multi_hand_landmarks):    # For every landmark point
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)    # Draw the corresponding joint and connection

In [ ]:
# Testing the Webcam Stream

cap = cv2.VideoCapture(0)    # Webcam stream component

# Using the mediapipe hands model
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands:
    while cap.isOpened():
        ret, frame = cap.read() # Get a frame from webcam stream

        # Detections
        image, results = hand_detection(frame, hands)
        
        # Draw Landmarks
        draw_landmarks(image, results)

        cv2.imshow('Hand Tracking', image)    # Frame after applying the mediapipe drawings
        
        if cv2.waitKey(10) & 0xFF == ord('q'):    # Press 'Q' to exit
            break

cap.release()
cv2.destroyAllWindows()

## Extracting Keypoints from Mediapipe output

In [ ]:
def extract_keypoints(results):
    all_landmarks = []    # Array to store the flattened coordinates of the 21 hand joints
    if results.multi_hand_world_landmarks:
        for hand_landmarks in results.multi_hand_world_landmarks:
            for _, landmark in enumerate(hand_landmarks.landmark):
                test = np.array([landmark.x, landmark.y, landmark.z])
                all_landmarks.append(test)
    else:
        all_landmarks.append(np.zeros(21 * 3))    # Return array of shape (21,) with zeroes if hands are not detected
    all_landmarks = np.array(all_landmarks).flatten()
    return all_landmarks

## Setup Directories

In [ ]:
data_path = os.path.join('Data')
letters = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 
                    'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'])    # 'J' & 'Z' omitted as they are motion based signs
no_sequences = 25
# left hand = 25 && right hand = 25
# Collect data twice for both hands
sequence_length = 10    # Number of frames per video collected

In [ ]:
# Make Directories to store the numPy arrays
for letter in letters:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(data_path, letter, str(sequence)))
        except:
            pass

## Extract Data and store in directories

In [ ]:
cap = cv2.VideoCapture(0)

# Using the mediapipe hands model
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands:
    for letter in letters:
        flag = True
        for sequence in range(no_sequences):
            for frame_number in range(-1, sequence_length):
                
                ret, frame = cap.read() # Get a frame from webcam stram

                if frame_number == -1:
                    while(flag):
                        ret, frame = cap.read()
                        cv2.putText(frame, 'START Collection for letter \'{}\', press S.'.format(letter), (120, 200), 
                                cv2.FONT_HERSHEY_COMPLEX, 1,(0, 255, 0), 1, cv2.LINE_AA)
                        cv2.imshow('Frames Collection', frame)
                        if cv2.waitKey(10) & 0xFF == ord('s'):    # Press 'S' to start capturing data for the particular letter
                            flag = False
                    continue
                
                # Detections
                image, results = hand_detection(frame, hands)
                draw_landmarks(image, results)
                
                cv2.putText(image, 'Collecting for letter \'{}\' video number {}'.format(letter, sequence), (120, 200), 
                            cv2.FONT_HERSHEY_COMPLEX, 1,(0, 255, 0), 1, cv2.LINE_AA)
                cv2.imshow('Frames Collection', image)    # Frame after applying the mediapipe drawings

                # Extracting and saving numPy arrays                
                keypoints = extract_keypoints(results)
                
                npy_path = os.path.join(data_path, letter, str(sequence), str(frame_number))    # Path to store the array at
                np.save(npy_path, keypoints)    # Save arrays for individual frames
                    
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

cap.release()
cv2.destroyAllWindows()

## Pre-Processing data and Creating Labels

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label: num for num, label in enumerate(letters)}    # Create a label map to train the model on

In [ ]:
# Merge labels with the corresponding sequence data
sequences, labels = [], []
for letter in letters:
    for sequence in range(no_sequences):
        window = []
        for frame_number in range(sequence_length):
            res = np.load(os.path.join(data_path, letter, str(sequence), "{}.npy".format(frame_number)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[letter])

In [ ]:
np.array(sequences).shape    # (total_videos, frames_per_video, keypoints_per_frame)

In [ ]:
X = np.array(sequences)    # The coordinate data to train the model with

In [ ]:
X.shape    # (a, b, c) => (b, c) is input shape for LSTM layer

In [ ]:
y = to_categorical(labels).astype(int)    # Labels for the corresponsing X elements e.g., X[0] => y[0] : {'a': 0}

In [ ]:
# Split the dataset into training (95%) and testing (5%) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Building and Training the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)    # (Optional) Directory for evaluating model with TensorBoard

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(10, 63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(letters.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Load the pre-trained model (h5 or keras format). Use only one!
# model = tensorflow.keras.models.load_model('asl.keras')
model.load_weights('asl.h5')

In [ ]:
# !!!! Do not run if model is already trained !!!!
# Train the model with the dataset, change epochs based on size of dataset
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

## Making Predictions and Saving Model

In [ ]:
res = model.predict(X_test)

In [ ]:
# Testing - Check if the model can predict the test split accurately
for i in range(30):
    print(i, ":")
    print(letters[np.argmax(res[i])] == letters[np.argmax(y_test[i])])

In [ ]:
model.save('asl.h5')
model.save('asl.keras')

## Testing the Model

In [ ]:
sequence = []
threshold = 0.7
res = np.array([0])
letter = ' '

cap = cv2.VideoCapture(0)

# Using the mediapipe hands model
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands:
    while cap.isOpened():
        ret, frame = cap.read() # Get a frame from webcam stram

        # Detections
        image, results = hand_detection(frame, hands)
        
        # Draw Landmarks
        draw_landmarks(image, results)

        # Testing Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-10:]

        if len(sequence) == 10:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(letters[np.argmax(res)])

        # Rendering result in OpenCV
        if res[np.argmax(res)] > threshold:
            letter = letters[np.argmax(res)]

        cv2.rectangle(image, (0,0), (640, 40), (255, 255, 255), -1)
        cv2.putText(image, ''.join(letter), (3, 30),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.imshow('Hand Tracking', image)    # Frame after applying the mediapipe drawings
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()